In [22]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [23]:
feature_list = ['x','y','hour','weekday','day','month','year','accuracy']

In [24]:
def prepare_data(df, n_cell_x, n_cell_y):
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_x'] = pos_x
    df['grid_y'] = pos_y
    
    #Feature engineering
    
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    df.accuracy = df.accuracy.values * fw[7]
    df = df.drop(['time'], axis=1)
    
    return df

In [25]:
def run_prediction(df_train, df_test, th):
    total_result = pd.DataFrame()
    for x_grid in range(0, n_cell_x):
        start_time = time.time()
        for y_grid in range(0, n_cell_y):
            total_result = total_result.append(process_one_cell(df_train, df_test, x_grid, y_grid, th))
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_grid, flush = True)
    return total_result

In [26]:
def process_one_cell(df_train, df_test, grid_x, grid_y, th):
    start_time = time.time()
    
    df_cell_train = df_train[(df_train.grid_x == grid_x)&(df_train.grid_y == grid_y)]
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    df_cell_test = df_test[(df_test.grid_x == grid_x)&(df_test.grid_y == grid_y)]
    row_ids = df_cell_test.row_id
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=25, weights='distance', 
                               metric='manhattan')
    clf.fit(df_cell_train[feature_list], df_cell_train.place_id)
    predictions = clf.predict_proba(df_cell_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_cell_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [27]:
def run_validation():
    # Run validation
    df_train = pd.read_csv('../train.csv',
                           usecols=['row_id','x','y','accuracy','time','place_id'])
    
    df_valiation = df_train[df_train.time > 786239 * 0.875]
    
    df_train = df_train[df_train.time <= 786239 * 0.875]
    
    n_cell_x = 10
    n_cell_y = 20
    
    print('Preparing train data')
    df_train = prepare_data(df_train, n_cell_x, n_cell_y)
    df_valiation = prepare_data(df_valiation, n_cell_x, n_cell_y)
    df_valiation['p1'] = np.nan
    df_valiation['p2'] = np.nan
    df_valiation['p3'] = np.nan
    
    prediction_result = run_prediction(df_train, df_valiation, 5)
    
    # Calculate score
    prediction_result.sort_index(inplace=True)
    prediction_result['score'] = (prediction_result.p1 == df_valiation.place_id) * 1
    prediction_result['score'] += (prediction_result.p2 == df_valiation.place_id) * 0.5
    prediction_result['score'] += (prediction_result.p3 == df_valiation.place_id) * 0.33
    
    print(prediction_result.score.describe(), flush=True)

In [28]:
#     x    y     hour weekday day     month year accuracy
fw = [500, 1000, 4,   3,      1./22., 2,    10,  0.0]
run_validation()

#     x    y     hour weekday day     month year accuracy
fw = [500, 1000, 4,   3,      1./22., 2,    10,  0.04]
run_validation()

#     x    y     hour weekday day     month year accuracy
fw = [500, 1000, 4,   3,      1./22., 2,    10,  0.06]
run_validation()

#     x    y     hour weekday day     month year accuracy
fw = [500, 1000, 4,   3,      1./22., 2,    10,  0.08]
run_validation()

#     x    y     hour weekday day     month year accuracy
fw = [500, 1000, 4,   3,      1./22., 2,    10,  0.2]
run_validation()

#     x    y     hour weekday day     month year accuracy
fw = [500, 1000, 4,   3,      1./22., 2,    10,  0.4]
run_validation()

#     x    y     hour weekday day     month year accuracy
fw = [500, 1000, 4,   3,      1./22., 2,    10,  0.6]
run_validation()

Preparing train data
Elapsed time overall: 49.54033851623535 seconds 0
Elapsed time overall: 58.99756741523743 seconds 1
Elapsed time overall: 69.34994983673096 seconds 2
Elapsed time overall: 61.68088626861572 seconds 3
Elapsed time overall: 62.92969846725464 seconds 4
Elapsed time overall: 50.83010220527649 seconds 5
Elapsed time overall: 56.40828800201416 seconds 6
Elapsed time overall: 62.560713052749634 seconds 7
Elapsed time overall: 57.511470794677734 seconds 8
Elapsed time overall: 51.65808057785034 seconds 9
count    4417684.000000
mean           0.533805
std            0.453489
min            0.000000
25%            0.000000
50%            0.500000
75%            1.000000
max            1.000000
Name: score, dtype: float64
Preparing train data
Elapsed time overall: 57.8394570350647 seconds 0
Elapsed time overall: 61.598270654678345 seconds 1
Elapsed time overall: 66.96595001220703 seconds 2
Elapsed time overall: 62.752609968185425 seconds 3
Elapsed time overall: 64.0343077182

accuracy weight, score <br/>
0      0.533805 <br/>
0.04   0.537554 <br/>
0.06   0.538640 <br/>
0.08   0.539371 <br/>
0.2    0.538641 <br/>
0.4    0.533665 <br/>
0.6    0.529443 <br/>

In [ ]:
# Write to file
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('validation_result.csv', index=False)

In [6]:
# Run test
df_train = pd.read_csv('../train.csv',
                       usecols=['row_id','x','y','accuracy','time','place_id'])
df_test = pd.read_csv('../test.csv',
                       usecols=['row_id','x','y','accuracy','time'])
n_cell_x = 10
n_cell_y = 20 

df_train = prepare_data(df_train, n_cell_x, n_cell_y)
df_test = prepare_data(df_test, n_cell_x, n_cell_y)
df_test['p1'] = np.nan
df_test['p2'] = np.nan
df_test['p3'] = np.nan
prediction_result = run_prediction(df_train, df_test, 5)

prediction_result.sort_index(inplace=True)
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('result.csv', index=False)

Elapsed time overall: 90.63769316673279 seconds 0
Elapsed time overall: 95.83221125602722 seconds 1
Elapsed time overall: 107.47481536865234 seconds 2
Elapsed time overall: 101.15630769729614 seconds 3
Elapsed time overall: 101.53010082244873 seconds 4
Elapsed time overall: 105.17719149589539 seconds 5
Elapsed time overall: 99.80930209159851 seconds 6
Elapsed time overall: 105.55863928794861 seconds 7
Elapsed time overall: 100.55230617523193 seconds 8
Elapsed time overall: 91.66067624092102 seconds 9
